In [27]:
import os
os.chdir("/Users/Layne/Desktop/aiqc")

In [28]:
import aiqc
from aiqc import examples

In [29]:
from importlib import reload
aiqc.delete_db(True)
reload(aiqc)
aiqc.create_db()


=> Success - deleted database file at path:
/Users/layne/Library/Application Support/aiqc/aiqc.sqlite3


=> Success - created database file for machine learning metrics at path:
/Users/layne/Library/Application Support/aiqc/aiqc.sqlite3


=> Success - created the following tables within database:
['algorithm', 'batch', 'datapipeline', 'dataset', 'experiment', 'featureset', 'file', 'fold', 'foldset', 'hyperparamcombo', 'hyperparamset', 'image', 'job', 'jobset', 'label', 'preprocess', 'result', 'splitset', 'tabular']



---

In [30]:
df = examples.demo_file_to_pandas('iris.tsv')
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [31]:
dataset = aiqc.Dataset.Tabular.from_pandas(df)

In [32]:
label = dataset.make_label(columns=['species'])

In [33]:
featureset = dataset.make_featureset(exclude_columns=['species'])

In [34]:
splitset = featureset.make_splitset(
    label_id = label.id
    , size_test = 0.22
    , size_validation = 0.12
)

In [35]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [36]:
encoder_features = StandardScaler()

In [37]:
encoder_labels = OneHotEncoder(sparse=False)

In [38]:
preprocess = splitset.make_preprocess(
    encoder_features = encoder_features
    , encoder_labels = encoder_labels
)

In [39]:
def function_model_build(**hyperparameters):
    model = Sequential()
    model.add(Dense(13, input_shape=(4,), activation='relu', kernel_initializer='he_uniform', name='fc1')) # first hidden layer
    model.add(Dropout(0.2))
    model.add(Dense(hyperparameters['l2_neuron_count'], activation='relu', kernel_initializer='he_uniform', name='fc2'))
    model.add(Dense(3, activation='softmax', name='output'))

    model.compile(
        loss='categorical_crossentropy'
        , optimizer=hyperparameters['optimizer']
        , metrics=['accuracy']
    )
    return model

In [40]:
def function_model_train(model, samples_train, samples_evaluate=None, **hyperparameters):
    model.fit(
        samples_train["features"]
        , samples_train["labels"]
        , validation_data = (
            samples_evaluate["features"]
            , samples_evaluate["labels"]
        )
        , verbose = 0
        , batch_size = 3
        , epochs = hyperparameters['epochs']
        , callbacks=[History()]
    )
    return model

In [41]:
samples = {}

In [42]:
samples['evaluate'] = None

In [43]:
hyperparameters = {
    "l2_neuron_count": [9, 13, 18]
    , "optimizer": ["adamax", "adam"]
    , "epochs": [66]
}

### Stage the Experiment and Parameters

In [44]:
algorithm = aiqc.Algorithm.make(
    library = "Keras"
    , analysis_type = "classification_multi"
	, function_model_build = function_model_build
	, function_model_train = function_model_train
)

In [45]:
hyperparamset = algorithm.make_hyperparamset(
	hyperparameters = hyperparameters
)

In [46]:
batch = algorithm.make_batch(
    splitset_id = splitset.id
    , hyperparamset_id = hyperparamset.id
    , preprocess_id = preprocess.id
    , hide_test = True
)

In [47]:
import keras
from keras import metrics
from keras.models import Sequential, load_model
from keras.callbacks import History
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier

In [48]:
batch.run_jobs()

🔮 Training Models 🔮: 100%|██████████████████████████████████████████| 6/6 [00:20<00:00,  3.34s/it]


In [52]:
batch.jobs[1].results[0].metrics

{'validation': {'accuracy': 0.8333333333333334,
  'f1': 0.8222222222222223,
  'loss': 0.2937889099121094,
  'precision': 0.8888888888888888,
  'recall': 0.8333333333333334,
  'roc_auc': 0.9814814814814815},
 'train': {'accuracy': 0.98989898989899,
  'f1': 0.9898966704936855,
  'loss': 0.082026407122612,
  'precision': 0.9901960784313726,
  'recall': 0.98989898989899,
  'roc_auc': 0.9996939087848179}}